In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pydantic import BaseModel,Field
from typing import List,Annotated,Optional
import operator

class User(BaseModel):
    name : str = Field(description="name of the organziation")
    logo : str = Field(description="Logo of the given website")
    description : str = Field(description='A detailed description of what the organization does ')
    services: List[str] = Field(description="A list of services offered by the organization on the given website")


class State(BaseModel):
    buyer : Optional[User] = Field(description=' ')
    seller :Optional[User] = Field(description=' ')
    client_requirement: str  
    additional_info : List[str]
    service_dept : str
    sections : List[str] = Field(description="Total sections inside the sales proposal")
    final_result : str = Field(description='')
    

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-flash')

In [4]:
def clean_to_list(result:str) :
    result = result.strip()
    if result.startswith('```python'):
        result = result[len('```python'):].strip()
    elif result.startswith('```json'):
        result = result[len('```json'):].strip()
    elif result.startswith('```'):
        result = result[len('```'):].strip()
    if result.endswith('```'):
        result = result[:-3].strip()
    return result

In [66]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# from ..utils import clean_to_list

template = '''You are a business consultant specialized in drafting professional sales proposals.

Based on the following services offered by the client, generate a list of high-level sections that should be included in a compelling business proposal. These sections should be relevant to the domain and type of services offered.

### Services:
{services}

Return only a clean list of section titles, no explanations or numbering.

IMPORTANT :
Dont put too much sections just include the important ones like the most necessary and not to forget Scope of work / Project breakdown 
'''
prompt = ChatPromptTemplate.from_template(template)


chain = prompt | llm | StrOutputParser()




def create_sections(state:State):
    result = chain.invoke({'services':state.service_dept})
    p = clean_to_list(result)
    return {'sections':["Title of the sales proposal"]+p.split('\n')}


In [65]:
s = 'Executive Summary\nIntroduction\nOur Approach\nTechnology Solutions\nScope of Work / Project Breakdown\nTimeline & Milestones\nPricing & Payment Terms\nClient Testimonials/Case Studies\nNext Steps/Call to Action\nAppendix (Optional)'
s.split('\n')

['Executive Summary',
 'Introduction',
 'Our Approach',
 'Technology Solutions',
 'Scope of Work / Project Breakdown',
 'Timeline & Milestones',
 'Pricing & Payment Terms',
 'Client Testimonials/Case Studies',
 'Next Steps/Call to Action',
 'Appendix (Optional)']

In [28]:
[1,2]+[3,4]

[1, 2, 3, 4]

In [67]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

p_template = '''You are a senior enterprise consultant and proposal strategist with years of experience writing persuasive, high-quality sales proposals for B2B companies across technology, digital, and marketing domains.

Your task is to generate a **structured, executive-grade Sales Proposal** based on the input below. Keep the tone professional and persuasive, suitable for leadership and decision-makers.

---

## 🧩 Buyer Organization:
{description_b}

## 🏢 Seller Organization:
{description_s}

## 📋 Client Requirements (Project Needs, Expectations, Goals, Constraints):
{client_requirements}

## 🎯 Target Department:
{service_dept}

---

### 📄 Required Sections

Use the following sections in this exact order. Do **not create new sections**, and **do not rename them**. Keep most sections **concise (2–4 paragraphs)** but make **“Scope of Work” very detailed** with structured breakdowns.

{section_list}

At the end, always include:
- **Who We Are**
- **What We Do**

---

### 🛠️ Special Instructions for “Scope of Work”

This section must be written in **detailed breakdown format**.

- Use clear subheaders or bullet points for each phase or deliverable.
- Elaborate tasks clearly using context from additional informtation
-----------
{additional_info}.
----------------
IMPORTANT : If there is no breakdown of the task then breakdown yourself and then explain each phase with atleast 4-5 bullet points
and if there is breakdown then do try to utilize info and still write 4-5 points properly
- Example format:

Discovery & Planning (Week 1–2)

Conduct kickoff with buyer team to gather expectations

Create user journeys and goal mapping

Perform competitive audit for design inspiration and SEO alignment

UI/UX Design (Week 3–4)

Wireframes for key pages

Brand-aligned visual mockups with review cycles

Frontend Development (Week 5–7)

Responsive development using Bootstrap

Animations, product pages, and forms



Use **phase-wise structuring** like Discovery, Design, Development, Launch, SEO, etc., and explain each step in business-impact terms — showing **how it contributes to client goals**.

---

### ✍️ Writing Style & Examples

Follow these guidelines:

- ✅ Use a **professional, business-friendly tone** suitable for executive audiences.
- ✅ Write **2–4 paragraphs per section** — detailed, but not bloated.
- ✅ Use bullet points where appropriate (for Deliverables, Timelines, Fee Structure).
- ✅ Integrate `{client_requirements}` contextually — refer to business goals, constraints, project scope, and domain.
- ✅ Emphasize **value, clarity, and confidence** — this is a persuasive sales proposal.

---

### 📤 Output Format

Return the final result as a list of dictionaries, each with `"title"` and `"text"` fields, in **valid JSON-like format**:

```json
[ {{"title":"Title of the proposal","text":"..."}}
  {{"title": "Executive Summary", "text": "..."}},
  {{"title": "Scope of Work", "text": "..."}},
  ...
  {{"title": "Who We Are", "text": "..."}},
  {{"title": "What We Do", "text": "..."}}
]

'''

# Set up LangChain prompt + parser
p_prompt = ChatPromptTemplate.from_template(p_template)
p_chain = p_prompt | llm | StrOutputParser()

# Function to generate the full proposal
def write_sales_proposal(state):
    section_string = "\n".join([f"- {s}" for s in state.sections])
    client_reqs = "\n".join(state.additional_info)

    result = p_chain.invoke({
        'description_s': state.seller.description,
        'description_b': state.buyer.description,
        'client_requirements': client_reqs,
        'service_dept': state.service_dept,
        'section_list': section_string,
        'additional_info':state.additional_info
    })

    return {'final_result': result}


In [68]:
from langgraph.graph import StateGraph,START,END

builder = StateGraph(State)
builder.add_node(create_sections)
builder.add_node(write_sales_proposal)

builder.add_edge(START,'create_sections')
builder.add_edge('create_sections','write_sales_proposal')
builder.add_edge('write_sales_proposal',END)

In [69]:
graph = builder.compile()

In [70]:
buyer = User(name='GROWTHSUTRA' ,logo='https://static.wixstatic.com/media/cb6b3d_5c8f2b020ebe48b69bc8c163cc480156~mv2.png/v1/fill/w_60,h_60,al_c,q_85,usm_0.66_1.00_0.01,enc_avif,quality_auto/GrowthSutra%20Logo.png' ,description="At GrowthSutra, we are the go-to experts dedicated to accelerate brand and revenue growth for startups and SMBs. We provide the Fortune 500-caliber strategic thinking and flawless execution needed to gain market access, customer traction, and investor interest. Our team is comprised of seasoned marketing, communications, sales, and leadership executives, each with over 20+ years of real-world expertise launching and scaling disruptive brands across technology, e-commerce, climate, FMCG, and other major industries. We combine our team\\'s unmatched experience with rigorous project governance and proven data-driven frameworks to get you measurable results fast. Our approach is tailored to each client\\'s unique needs and objectives." ,services=['XPRT Co-Pilots', 'On Demand CMO', 'Ice Breakers', 'GTM Partner', 'Go-To-Market Partner', 'Revenue Architect', 'On-Demand CMO'])

In [71]:
buyer.logo

'https://static.wixstatic.com/media/cb6b3d_5c8f2b020ebe48b69bc8c163cc480156~mv2.png/v1/fill/w_60,h_60,al_c,q_85,usm_0.66_1.00_0.01,enc_avif,quality_auto/GrowthSutra%20Logo.png'

In [72]:
seller = User(name = 'TCS', logo = 'https://www.tcs.com/content/dam/global-tcs/en/images/home/tata-logo-1.svg',description= 'TCS works hand in hand with world-leading investors.', services=  ['Artificial Intelligence', 'Cloud', 'Cognitive Business Operations', 'Consulting', 'Cybersecurity', 'Data & Analytics', 'Enterprise' ])

In [73]:
client_requirement = "Building a website for their domain"

additonal_info = ["This task is done in like 2 weeeks with cost of $4000 "]

service_dept = 'Technology'

In [74]:
state = State(buyer = seller,seller = buyer,client_requirement=client_requirement,additional_info=additonal_info,service_dept=service_dept,sections=[],final_result='')

In [75]:
# create_sections(state)

In [76]:
state = graph.invoke(state)

In [77]:
print(state['final_result'])

```json
[
  {"title": "Title of the sales proposal", "text": "Accelerating Brand & Revenue Growth for TCS: A GrowthSutra Proposal"},
  {"title": "Executive Summary", "text": "GrowthSutra proposes a two-week, $4,000 engagement to significantly enhance TCS's brand visibility and investor appeal within the technology sector. Leveraging our team's decades of experience in launching and scaling disruptive brands, we will deliver a targeted strategy focused on achieving measurable results within the stipulated timeframe. This proposal outlines a comprehensive plan encompassing strategic planning, brand messaging refinement, and targeted outreach to key stakeholders, ensuring alignment with TCS's overall objectives and investor expectations."},
  {"title": "Introduction", "text": "TCS, a leader in collaborating with world-class investors, requires a rapid and impactful solution to amplify its brand presence and attract further investment. GrowthSutra understands this need and offers a tailore

In [78]:
def clean_to_list(result:str) :
    result = result.strip()
    if result.startswith('```python'):
        result = result[len('```python'):].strip()
    elif result.startswith('```json'):
        result = result[len('```json'):].strip()
    elif result.startswith('```'):
        result = result[len('```'):].strip()
    if result.endswith('```'):
        result = result[:-3].strip()
    return result

result = clean_to_list(state['final_result'])

In [79]:
import json
result = json.loads(result)

In [80]:
print(result)

[{'title': 'Title of the sales proposal', 'text': 'Accelerating Brand & Revenue Growth for TCS: A GrowthSutra Proposal'}, {'title': 'Executive Summary', 'text': "GrowthSutra proposes a two-week, $4,000 engagement to significantly enhance TCS's brand visibility and investor appeal within the technology sector. Leveraging our team's decades of experience in launching and scaling disruptive brands, we will deliver a targeted strategy focused on achieving measurable results within the stipulated timeframe. This proposal outlines a comprehensive plan encompassing strategic planning, brand messaging refinement, and targeted outreach to key stakeholders, ensuring alignment with TCS's overall objectives and investor expectations."}, {'title': 'Introduction', 'text': 'TCS, a leader in collaborating with world-class investors, requires a rapid and impactful solution to amplify its brand presence and attract further investment. GrowthSutra understands this need and offers a tailored, results-driv

In [81]:
with open('output.txt', 'w', encoding='utf-8') as f:
    for entry in result:
        f.write(f"Title: {entry['title']}\n")
        f.write(f"Text: {entry['text']}\n\n")

In [6]:

import pdfkit

def convert_html_to_pdf(html_path, output_pdf=None):
    if output_pdf is None:
        output_pdf = html_path.replace('.html', '.pdf')
    
    options = {
        'enable-local-file-access': None,
        'page-size': 'A4',
        'encoding': 'UTF-8',
        'quiet': ''
    }
    
    pdfkit.from_file(html_path, output_pdf, options=options)
    return output_pdf


In [4]:
html_path = '/home/shreyank/Gen-ai/Growth/PresesntationWriting/output_presentation.html'

In [7]:
convert_html_to_pdf(html_path)

'/home/shreyank/Gen-ai/Growth/PresesntationWriting/output_presentation.pdf'

In [5]:
def html_to_pdf_pdfkit(html_content, output_path):
    """
    pdfkit with specific options to handle page breaks
    pip install pdfkit
    Also need to install wkhtmltopdf system package
    """
    try:
        import pdfkit
        
        options = {
            'page-size': 'A4',
            'margin-top': '20mm',
            'margin-right': '20mm',
            'margin-bottom': '20mm',
            'margin-left': '20mm',
            'encoding': "UTF-8",
            'no-outline': None,
            'enable-local-file-access': None,
            'print-media-type': None,
            'disable-smart-shrinking': None,
        }
        
        pdfkit.from_string(html_content, output_path, options=options)
        print(f"PDF generated successfully: {output_path}")
        return True
    except ImportError:
        print("pdfkit not installed. Run: pip install pdfkit")
        return False
    except Exception as e:
        print(f"Error generating PDF: {e}")
        return False


html_to_pdf_pdfkit(html_path,'output.pdf')

PDF generated successfully: output.pdf


True